<a href="https://colab.research.google.com/github/pavelmajek/fcc-machine-learning-with-python/blob/main/neural_network_sms_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep="\t", names=["ham", "text"])
test_data = pd.read_csv(test_file_path, sep="\t", names=["ham", "text"])
train_label = train_data.pop("ham")
test_label = test_data.pop("ham")
train_labels = np.array([0 if i == "ham" else 1 for i in train_label])
test_labels = np.array([0 if i == "ham" else 1 for i in test_label])

In [ ]:
import matplotlib.pyplot as plt

histogram = pd.DataFrame()
histogram["text"] = train_data["text"].apply(len)

plt.hist(histogram["text"], bins=20, color='blue', alpha=0.7)
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of ' + "text")
plt.grid(True)
plt.show()

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, one_hot

MAXLEN = 50
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'].values)

encoded_train = tokenizer.texts_to_sequences(train_data['text'].values)
encoded_test = tokenizer.texts_to_sequences(test_data['text'].values)

padded_train = pad_sequences(encoded_train, maxlen=MAXLEN, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=MAXLEN, padding='post')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten, Dropout, Embedding, LSTM
model = Sequential([
    Embedding(50000, 24, input_length=MAXLEN),
    LSTM(24, return_sequences=True),
    SimpleRNN(24, return_sequences=False),
    Dense(1, activation='sigmoid')
    ])

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics='accuracy')
history = model.fit(padded_train, train_labels, epochs=10, verbose=1, validation_data=[padded_test, test_labels])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_encoded = tokenizer.texts_to_sequences([pred_text])
  pred_padded = pad_sequences(pred_encoded, maxlen=MAXLEN, padding='post')
  prediction = model.predict(pred_padded).flatten()[0]

  if prediction < 0.5:
    return (prediction, 'ham')
  return (prediction, 'spam')

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
